In [57]:
import numpy as np
import pandas as pd

In [58]:
tcr_ab_pairs = pd.read_table('data/tcr_ab_pairs.txt')
tcr_ab_pairs.head()

,sample,clone,nSeqCDR3,aaSeqCDR3,allVGenes,allDGenes,allJGenes,positionOfCDR3Begin,positionOfVEndTrimmed,positionOfDBeginTrimmed,positionOfDEndTrimmed,positionOfJBeginTrimmed,chain
0,1,0,TGTGCAGCATTAGGTGGTTCTGCAAGGCAACTGACCTTT,CAALGGSARQLTF,TRAV29DV5,NaN,TRAJ22,48,57,-1,-1,62,A
1,1,0,TGTGCCAGCAGTTACGGAGCCCCCGGGACAGCCTTTTACGAGCAGT...,CASSYGAPGTAFYEQYF,"TRBV6-5,TRBV6-8,TRBV6-6,TRBV6-3,TRBV6-2","TRBD1,TRBD2",TRBJ2-7,36,51,57,67,72,B
2,1,2,TGTGCTGTGACCGATAGCAACTATCAGTTAATCTGG,CAVTDSNYQLIW,TRAV1-2,NaN,TRAJ33,51,61,-1,-1,63,A
3,1,2,TGCGCCAGCAGCCAAGATCGAGCGGGAGGGCCGGATACGCAGTATTTT,CASSQDRAGGPDTQYF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD2,TRBJ2-3,39,58,59,71,72,B
4,1,3,TGTGCTGTCACCCCCTTCAAAGCTGCAGGCAACAAGCTAACTTTT,CAVTPFKAAGNKLTF,TRAV41,NaN,TRAJ17,42,52,-1,-1,58,A


In [59]:
def MergeByClonotype(df, clone_col, chain, alpha_chain):
    clones_set = set(df[clone_col])
    clones_ab_presence = {x:[False, False] for x in clones_set}
    for clone in df[clone_col][df[chain] == alpha_chain]:
        clones_ab_presence[clone][0] = True
    for clone in df[clone_col][df[chain] != alpha_chain]:
        clones_ab_presence[clone][1] = True
    
    clones = set([])
    for clone in clones_set:
        if np.all(clones_ab_presence[clone]):
            clones.add(clone)

    clones_dict = {x:i for i, x in enumerate(clones)}
    
    df_array = df.as_matrix()
    data = np.zeros((len(clones), 2 * df.shape[1]), dtype=object)
    for i in range(df_array.shape[0]):
        clone =  df[clone_col][i]
        if clone in clones:
            index = clones_dict[clone]
            if df[chain][i] == alpha_chain:
                data[index, :df.shape[1]] = df_array[i, :]
            else:
                data[index, df.shape[1]:] = df_array[i, :]
    columns = ['alpha.' + x for x in df.columns] + ['beta.' + x for x in df.columns]
    return pd.DataFrame(data, columns=columns)

In [60]:
tcr_ab_pairs_pd = MergeByClonotype(tcr_ab_pairs, 'clone', 'chain', 'A')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [61]:
vdjdb = pd.read_table('data/vdjdb_full.txt')

x = pd.notnull(vdjdb.iloc[:, 0])
y = pd.notnull(vdjdb.iloc[:, 3])
valid = [x[i] and y[i] for i in range(vdjdb.shape[0])]
vdjdb = vdjdb.iloc[valid, :].drop_duplicates()
vdjdb.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cdr3.alpha,v.alpha,j.alpha,cdr3.beta,v.beta,d.beta,j.beta,species,mhc.a,mhc.b,...,meta.replica.id,meta.clone.id,meta.epitope.id,meta.tissue,meta.donor.MHC,meta.donor.MHC.method,meta.structure.id,cdr3fix.alpha,cdr3fix.beta,vdjdb.score
142,CAGNNARPMF,TRAV27*01,TRAJ31*01,CASSQSPGGVAFF,TRBV14*01,NaN,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,B2M,...,15days,259,20788,PBMC,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARPMF"", ""cdr3_old"": ""YLCAGNNARP...","{""cdr3"": ""CASSQSPGGVAFF"", ""cdr3_old"": ""YFCASSQ...",0
143,CAGNNARLMF,TRAV27*01,TRAJ31*01,CASSQSPGGVAFF,TRBV14*01,NaN,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,B2M,...,15days,"18,1.4,4.3",20788,PBMC,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARLMF"", ""cdr3_old"": ""YLCAGNNARL...","{""cdr3"": ""CASSQSPGGVAFF"", ""cdr3_old"": ""YFCASSQ...",0
144,CAGNNARLVF,TRAV27*01,TRAJ31*01,CASSQSPGGTQYF,TRBV14*01,NaN,TRBJ2-5*01,HomoSapiens,HLA-A*02:01,B2M,...,15days,"36,2.2",20788,PBMC,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARLVF"", ""cdr3_old"": ""YLCAGNNARL...","{""cdr3"": ""CASSQSPGGTQYF"", ""cdr3_old"": ""YFCASSQ...",0
145,CAEGFASSDGQKLLF,TRAV5*01,NaN,CASSQSPGGTQYF,TRBV14*01,NaN,TRBJ2-5*01,HomoSapiens,HLA-A*02:01,B2M,...,15days,186,20788,PBMC,NaN,NaN,NaN,"{""cdr3"": ""CAEGFASSDGQKLLF"", ""cdr3_old"": ""YFCAE...","{""cdr3"": ""CASSQSPGGTQYF"", ""cdr3_old"": ""YFCASSQ...",0
146,CAGNNARLMF,TRAV27*01,TRAJ31*01,CSARDRHREWLHF,TRBV20-1*01,NaN,TRBJ1-2*01,HomoSapiens,HLA-A*02:01,B2M,...,15days,2.3,20788,PBMC,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARLMF"", ""cdr3_old"": ""YLCAGNNARL...","{""cdr3"": ""CSARDRHREWLHF"", ""cdr3_old"": ""YICSARD...",0


In [62]:
vdjdb_ab = pd.read_table('data/VDJDB_full_ab.txt')

x = pd.notnull(vdjdb_ab.iloc[:, 0])
y = pd.notnull(vdjdb_ab.iloc[:, 3])
valid = [x[i] and y[i] for i in range(vdjdb_ab.shape[0])]
vdjdb_ab = vdjdb_ab.iloc[valid, :].drop_duplicates()
vdjdb_ab.head()

,cdr3.alpha,v.alpha,j.alpha,cdr3.beta,v.beta,d.beta,j.beta,species,mhc.a,mhc.b,...,meta.donor.MHC,meta.donor.MHC.method,meta.structure.id,cdr3fix.alpha,cdr3fix.beta,vdjdb.score,genP_1mism_alpha,genP_0mism_alpha,genP_1mism_beta,genP_0mism_beta
78,CAGNNARPMF,TRAV27*01,TRAJ31*01,CASSQSPGGVAFF,TRBV14*01,NaN,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,B2M,...,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARPMF"", ""cdr3_old"": ""YLCAGNNARP...","{""cdr3"": ""CASSQSPGGVAFF"", ""cdr3_old"": ""YFCASSQ...",0,1.450533e-05,0.000000,6.700000e-08,1.000000e-09
79,CAGNNARLMF,TRAV27*01,TRAJ31*01,CASSQSPGGVAFF,TRBV14*01,NaN,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,B2M,...,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARLMF"", ""cdr3_old"": ""YLCAGNNARL...","{""cdr3"": ""CASSQSPGGVAFF"", ""cdr3_old"": ""YFCASSQ...",0,1.621160e-04,0.000015,6.700000e-08,1.000000e-09
80,CAGNNARLVF,TRAV27*01,TRAJ31*01,CASSQSPGGTQYF,TRBV14*01,NaN,TRBJ2-5*01,HomoSapiens,HLA-A*02:01,B2M,...,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARLVF"", ""cdr3_old"": ""YLCAGNNARL...","{""cdr3"": ""CASSQSPGGTQYF"", ""cdr3_old"": ""YFCASSQ...",0,1.450600e-05,0.000000,7.816667e-07,1.300000e-08
81,CAEGFASSDGQKLLF,TRAV5*01,NaN,CASSQSPGGTQYF,TRBV14*01,NaN,TRBJ2-5*01,HomoSapiens,HLA-A*02:01,B2M,...,NaN,NaN,NaN,"{""cdr3"": ""CAEGFASSDGQKLLF"", ""cdr3_old"": ""YFCAE...","{""cdr3"": ""CASSQSPGGTQYF"", ""cdr3_old"": ""YFCASSQ...",0,3.000000e-09,0.000000,7.816667e-07,1.300000e-08
82,CAGNNARLMF,TRAV27*01,TRAJ31*01,CSARDRHREWLHF,TRBV20-1*01,NaN,TRBJ1-2*01,HomoSapiens,HLA-A*02:01,B2M,...,NaN,NaN,NaN,"{""cdr3"": ""CAGNNARLMF"", ""cdr3_old"": ""YLCAGNNARL...","{""cdr3"": ""CSARDRHREWLHF"", ""cdr3_old"": ""YICSARD...",0,1.621160e-04,0.000015,0.000000e+00,0.000000e+00


In [63]:
cd8 = pd.read_table('data/10x_48data/Cd8_healthyPBMC_TcellEnrich.txt')
cd8 = MergeByClonotype(cd8, 'clonotype_id', 'chain', 'TRA')
cd8.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


,alpha.clonotype_id,alpha.consensus_id,alpha.length,alpha.chain,alpha.v_gene,alpha.d_gene,alpha.j_gene,alpha.c_gene,alpha.full_length,alpha.productive,...,beta.c_gene,beta.full_length,beta.productive,beta.cdr3,beta.cdr3_nt,beta.reads,beta.umis,beta.total.TRA.contigs,beta.total.TRB.contigs,beta.total.contigs.num
0,clonotype748,clonotype748_consensus_1,538,TRA,TRAV17,None,TRAJ56,TRAC,True,True,...,TRBC1,True,True,CASTDPTGNQEKLFF,TGCGCCAGCACCGACCCCACGGGGAATCAGGAAAAACTGTTTTTT,15908,14,1,1,2
1,clonotype164,clonotype164_consensus_2,557,TRA,TRAV26-1,None,TRAJ22,TRAC,True,True,...,TRBC2,True,True,CASSQASPGLAGGSYEQYF,TGCGCCAGCAGCCAAGCCTCACCGGGACTAGCGGGAGGCTCCTACG...,2665,2,1,1,2
2,clonotype1058,clonotype1058_consensus_1,486,TRA,TRAV13-1,None,TRAJ30,TRAC,True,True,...,TRBC2,True,True,CASSLQGARETQYF,TGTGCCAGCAGCTTACAGGGGGCGCGAGAGACCCAGTACTTC,7181,7,1,1,2
3,clonotype639,clonotype639_consensus_1,469,TRA,TRAV35,None,TRAJ45,TRAC,True,True,...,TRBC2,True,True,CASSSSGLSQYF,TGTGCCAGCAGCTCGAGCGGGTTGTCGCAGTATTTT,5383,5,1,1,2
4,clonotype1210,clonotype1210_consensus_2,514,TRA,TRAV13-2,None,TRAJ24,TRAC,True,True,...,TRBC2,True,True,CASSQTSGAKNIQYF,TGTGCCAGCAGCCAAACTAGCGGAGCCAAAAACATTCAGTACTTC,2951,3,1,1,2


In [64]:
cd4 = pd.read_table('data/10x_48data/Cd4_healthyPBMC_TcellEnrich.txt')
cd4 = MergeByClonotype(cd4, 'clonotype_id', 'chain', 'TRA')
cd4.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


,alpha.clonotype_id,alpha.consensus_id,alpha.length,alpha.chain,alpha.v_gene,alpha.d_gene,alpha.j_gene,alpha.c_gene,alpha.full_length,alpha.productive,...,beta.c_gene,beta.full_length,beta.productive,beta.cdr3,beta.cdr3_nt,beta.reads,beta.umis,beta.total.TRA.contigs,beta.total.TRB.contigs,beta.total.contigs.num
0,clonotype748,clonotype748_consensus_1,538,TRA,TRAV17,None,TRAJ56,TRAC,True,True,...,TRBC1,True,True,CASTDPTGNQEKLFF,TGCGCCAGCACCGACCCCACGGGGAATCAGGAAAAACTGTTTTTT,15908,14,1,1,2
1,clonotype164,clonotype164_consensus_2,557,TRA,TRAV26-1,None,TRAJ22,TRAC,True,True,...,TRBC2,True,True,CASSQASPGLAGGSYEQYF,TGCGCCAGCAGCCAAGCCTCACCGGGACTAGCGGGAGGCTCCTACG...,2665,2,1,1,2
2,clonotype1058,clonotype1058_consensus_1,486,TRA,TRAV13-1,None,TRAJ30,TRAC,True,True,...,TRBC2,True,True,CASSLQGARETQYF,TGTGCCAGCAGCTTACAGGGGGCGCGAGAGACCCAGTACTTC,7181,7,1,1,2
3,clonotype639,clonotype639_consensus_1,469,TRA,TRAV35,None,TRAJ45,TRAC,True,True,...,TRBC2,True,True,CASSSSGLSQYF,TGTGCCAGCAGCTCGAGCGGGTTGTCGCAGTATTTT,5383,5,1,1,2
4,clonotype1210,clonotype1210_consensus_2,514,TRA,TRAV13-2,None,TRAJ24,TRAC,True,True,...,TRBC2,True,True,CASSQTSGAKNIQYF,TGTGCCAGCAGCCAAACTAGCGGAGCCAAAAACATTCAGTACTTC,2951,3,1,1,2


In [65]:
array_n = [0, tcr_ab_pairs_pd.shape[0], vdjdb.shape[0], vdjdb_ab.shape[0], cd4.shape[0], cd8.shape[0]]
for i in range(1, len(array_n)):
    array_n[i] += array_n[i - 1]

In [66]:
columns = ['database', 'species', 'sample', 
           'epitope', 'antigen', 'tissue', 'cell_subset', 
           'alpha.v', 'alpha.j', 'alpha.cdr3', 
           'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

data_pd = pd.DataFrame(np.nan * np.zeros((array_n[-1], len(columns)), dtype=object), columns=columns)

In [67]:
columns = ['database', 'species', 'sample', 
           'epitope', 'antigen', 'tissue', 'cell_subset', 
           'alpha.v', 'alpha.j', 'alpha.cdr3', 
           'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

data_col2n = {x:i for i,x in enumerate(columns)}

data = np.nan * np.zeros((array_n[-1], len(columns)), dtype=object)

In [68]:
data_i = 0

new_columns = ['database', 'species']
fill_with_values = ['pairseq', 'HomoSapiens']

for i, j in zip(new_columns, fill_with_values):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = j

old_columns = ['alpha.sample', 'alpha.allVGenes', 'alpha.allJGenes', 'alpha.aaSeqCDR3', 
               'beta.allVGenes', 'beta.allDGenes', 'beta.allJGenes','beta.aaSeqCDR3']
new_columns = ['sample', 'alpha.v', 'alpha.j', 'alpha.cdr3', 'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

for i, j in zip(new_columns, old_columns):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = tcr_ab_pairs_pd[j]

In [69]:
data_i = 1

new_columns = ['database']
fill_with_values = ['vdjdb']

for i, j in zip(new_columns, fill_with_values):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = j

old_columns = ['species', 'antigen.epitope', 'meta.epitope.id', 'meta.tissue', 'meta.cell.subset',
               'v.alpha', 'j.alpha', 'cdr3.alpha', 
               'v.beta', 'd.beta', 'j.beta', 'cdr3.beta']
new_columns = ['species', 'epitope', 'antigen', 'tissue', 'cell_subset',
               'alpha.v', 'alpha.j', 'alpha.cdr3', 'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

for i, j in zip(new_columns, old_columns):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = vdjdb[j]

In [70]:
data_i = 2

new_columns = ['database']
fill_with_values = ['vdjdb_ab']

for i, j in zip(new_columns, fill_with_values):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = j

old_columns = ['species', 'antigen.epitope', 'meta.epitope.id', 'meta.tissue', 'meta.cell.subset',
               'v.alpha', 'j.alpha', 'cdr3.alpha', 
               'v.beta', 'd.beta', 'j.beta', 'cdr3.beta']
new_columns = ['species', 'epitope', 'antigen', 'tissue', 'cell_subset',
               'alpha.v', 'alpha.j', 'alpha.cdr3', 'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

for i, j in zip(new_columns, old_columns):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = vdjdb_ab[j]

In [71]:
data_i = 3

new_columns = ['database', 'species', 'cell_subset']
fill_with_values = ['cd4', 'HomoSapiens', 'CD4']

for i, j in zip(new_columns, fill_with_values):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = j
    
old_columns = ['alpha.clonotype_id', 
               'alpha.v_gene', 'alpha.j_gene', 'alpha.cdr3', 
               'beta.v_gene', 'beta.d_gene', 'beta.j_gene', 'beta.cdr3']
new_columns = ['sample', 
               'alpha.v', 'alpha.j', 'alpha.cdr3', 
               'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

for i, j in zip(new_columns, old_columns):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = cd4[j]

In [72]:
data_i = 4

new_columns = ['database', 'species', 'cell_subset']
fill_with_values = ['cd8', 'HomoSapiens', 'CD8']

for i, j in zip(new_columns, fill_with_values):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = j
    
old_columns = ['alpha.clonotype_id', 
               'alpha.v_gene', 'alpha.j_gene', 'alpha.cdr3', 
               'beta.v_gene', 'beta.d_gene', 'beta.j_gene', 'beta.cdr3']
new_columns = ['sample', 
               'alpha.v', 'alpha.j', 'alpha.cdr3', 
               'beta.v', 'beta.d', 'beta.j', 'beta.cdr3']

for i, j in zip(new_columns, old_columns):
    data[array_n[data_i]:array_n[data_i + 1], data_col2n[i]] = cd8[j]

## duplication merging

In [73]:
cdr3 = set(data[:, data_col2n['alpha.cdr3']] + '_' + data[:, data_col2n['beta.cdr3']])
cdr3_ind = {x:[] for x in cdr3}
for i, x in enumerate(data[:, data_col2n['alpha.cdr3']] + '_' + data[:, data_col2n['beta.cdr3']]):
    cdr3_ind[x].append(i)

In [74]:
dm_data = np.nan * np.zeros((len(cdr3), data.shape[1]), dtype=object)

for i, cdr3_ab in enumerate(cdr3):
    indexes = cdr3_ind[cdr3_ab]
    idata = data[indexes, :]
    for c in range(data.shape[1]):
        values = set([str(x) for x in idata[:, c]])
        if 'nan' in values:
            values.remove('nan')
        if len(values) == 0:
            dm_data[i, c] = np.nan
        else:
            dm_data[i, c] = ','.join(values)

In [77]:
dm_data[dm_data[:, 0] == 'vdjdb_ab,vdjdb', 0] = 'vdjdb'
dm_data[dm_data[:, 0] == 'vdjdb_ab,vdjdb,pairseq', 0] = 'vdjdb,pairseq'

In [78]:
dm_data_pd = pd.DataFrame(dm_data, columns=columns)

In [79]:
set(dm_data_pd['database'])

{'cd8,cd4', 'pairseq', 'vdjdb', 'vdjdb,pairseq'}

In [80]:
dm_data_pd.to_csv('data/MergedData.csv')